In [9]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
import re

def extract_category_keywords(business_numbers):
    """
    주어진 사업자 번호 리스트를 사용하여 각 사업자 번호의 카테고리 키워드를 추출하는 함수.
    
    Args:
        business_numbers (list): 사업자 번호의 리스트.
    
    Returns:
        dict: 각 사업자 번호와 관련된 카테고리 키워드를 포함한 딕셔너리.
    """
    # Selenium driver 설정
    driver = webdriver.Chrome()

    # 카테고리 키워드를 저장할 딕셔너리
    category_keywords_dict = {}

    # 리스트에 있는 모든 사업자 번호에 대해 반복 실행
    for business_number in business_numbers:
        # 사업자 번호 형식에서 "-" 제거 (URL에 넣을 때는 숫자만 필요)
        business_number_clean = business_number.replace("-", "")
        
        # 동적으로 URL 생성
        address = 'https://bizno.net/article/' + business_number_clean
        print(f"접속 중인 URL: {address}")

        # URL로 이동
        driver.get(address)

        # category_keywords 추출
        try:
            shop_name = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/div[1]/div[1]/div/div[1]/div/a/h1').text
            print(f"사업자 번호 {business_number}의 상호명: {shop_name}")
            
            category_keywords = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/div[1]/div[1]/div/table/tbody/tr[2]/td').text
            print(f"사업자 번호 {business_number}의 업태: {category_keywords}")

            # 정규 표현식을 사용해 각 분류를 추출
            category_dict = {
                "대분류": re.search(r"대분류\s*:\s*(.*?)(?=\s*중분류|$)", category_keywords),
                "중분류": re.search(r"중분류\s*:\s*(.*?)(?=\s*소분류|$)", category_keywords),
                "소분류": re.search(r"소분류\s*:\s*(.*?)(?=\s*세분류|$)", category_keywords),
                "세분류": re.search(r"세분류\s*:\s*(.*?)(?=\s*세세분류|$)", category_keywords),
                "세세분류": re.search(r"세세분류\s*:\s*(.*)", category_keywords)
            }

            # None을 제거하고 텍스트만 추출
            for key in category_dict:
                if category_dict[key]:
                    category_dict[key] = category_dict[key].group(1).strip()
                else:
                    category_dict[key] = None

            category_keywords_dict[business_number] = category_dict
        except Exception as e:
            print(f"사업자 번호 {business_number}에 대한 정보를 찾을 수 없습니다. 오류: {e}")
            category_keywords_dict[business_number] = None

    # 드라이버 종료
    driver.quit()

    # 결과 반환
    return category_keywords_dict

def save_to_json(data, file_path):
    """
    데이터를 JSON 파일로 저장하는 함수.
    
    Args:
        data (dict): 저장할 데이터.
        file_path (str): 저장할 JSON 파일 경로.
    """
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
    print(f"데이터가 {file_path}에 JSON 형식으로 저장되었습니다.")
    

# 함수 호출 및 결과 출력
category_keywords = extract_category_keywords(business_numbers)

# 추출한 카테고리 키워드가 성공적으로 있으면 JSON 파일로 저장
if category_keywords:
    json_file_path = 'category_keywords.json'
    save_to_json(category_keywords, json_file_path)
else:
    print("카테고리 키워드가 추출되지 않아 JSON 파일로 저장되지 않았습니다.")


접속 중인 URL: https://bizno.net/article/6182103502
사업자 번호 618-21-03502의 상호명: 365열린약국
사업자 번호 618-21-03502의 업태: 대분류 : 도매 및 소매업
중분류 : 소매업; 자동차 제외
소분류 : 기타 상품 전문 소매업
세분류 : 의약품, 의료용 기구, 화장품 및 방향제 소매업
세세분류 : 의약품 및 의료용품 소매업
데이터가 category_keywords.json에 JSON 형식으로 저장되었습니다.


In [8]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from concurrent.futures import ThreadPoolExecutor

def extract_single_category(business_number):
    """
    단일 사업자 번호에 대한 카테고리 키워드를 추출하는 함수.
    """
    # Selenium driver 설정
    driver = webdriver.Chrome()

    # 사업자 번호 형식에서 "-" 제거 (URL에 넣을 때는 숫자만 필요)
    business_number_clean = business_number.replace("-", "")
    
    # 동적으로 URL 생성
    address = 'https://bizno.net/article/' + business_number_clean
    print(f"접속 중인 URL: {address}")

    # URL로 이동
    driver.get(address)

    # 카테고리 키워드를 저장할 딕셔너리
    category_dict = {}

    try:
        shop_name = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/div[1]/div[1]/div/div[1]/div/a/h1').text
        print(f"사업자 번호 {business_number}의 상호명: {shop_name}")
        
        category_keywords = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/div[1]/div[1]/div/table/tbody/tr[2]/td').text
        print(f"사업자 번호 {business_number}의 업태: {category_keywords}")

        # 정규 표현식을 사용해 각 분류를 추출
        category_dict = {
            "대분류": re.search(r"대분류\s*:\s*(.*?)(?=\s*중분류|$)", category_keywords),
            "중분류": re.search(r"중분류\s*:\s*(.*?)(?=\s*소분류|$)", category_keywords),
            "소분류": re.search(r"소분류\s*:\s*(.*?)(?=\s*세분류|$)", category_keywords),
            "세분류": re.search(r"세분류\s*:\s*(.*?)(?=\s*세세분류|$)", category_keywords),
            "세세분류": re.search(r"세세분류\s*:\s*(.*)", category_keywords)
        }

        # None을 제거하고 텍스트만 추출
        for key in category_dict:
            if category_dict[key]:
                category_dict[key] = category_dict[key].group(1).strip()
            else:
                category_dict[key] = None

    except Exception as e:
        print(f"사업자 번호 {business_number}에 대한 정보를 찾을 수 없습니다. 오류: {e}")
        category_dict = None

    # 드라이버 종료
    driver.quit()

    return {business_number: category_dict}

def extract_category_keywords(business_numbers):
    """
    주어진 사업자 번호 리스트를 사용하여 각 사업자 번호의 카테고리 키워드를 추출하는 함수.
    멀티스레딩을 사용하여 성능을 향상시킴.
    """
    # ThreadPoolExecutor를 사용해 병렬로 실행
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(extract_single_category, business_numbers))

    # 결과를 딕셔너리로 병합
    category_keywords_dict = {}
    for result in results:
        category_keywords_dict.update(result)

    return category_keywords_dict

def save_to_json(data, file_path):
    """
    데이터를 JSON 파일로 저장하는 함수.
    """
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
    print(f"데이터가 {file_path}에 JSON 형식으로 저장되었습니다.")

# 예시: 사업자 번호 리스트
business_numbers = ['618-21-03502']  # 여기에 실제 사업자 번호 리스트를 추가하세요.

# 함수 호출 및 결과 출력
category_keywords = extract_category_keywords(business_numbers)

# 추출한 카테고리 키워드가 성공적으로 있으면 JSON 파일로 저장
if category_keywords:
    json_file_path = 'category_keywords.json'
    save_to_json(category_keywords, json_file_path)
else:
    print("카테고리 키워드가 추출되지 않아 JSON 파일로 저장되지 않았습니다.")


접속 중인 URL: https://bizno.net/article/6182103502
사업자 번호 618-21-03502의 상호명: 365열린약국
사업자 번호 618-21-03502의 업태: 대분류 : 도매 및 소매업
중분류 : 소매업; 자동차 제외
소분류 : 기타 상품 전문 소매업
세분류 : 의약품, 의료용 기구, 화장품 및 방향제 소매업
세세분류 : 의약품 및 의료용품 소매업
데이터가 category_keywords.json에 JSON 형식으로 저장되었습니다.
